In [2]:
!wget https://raw.githubusercontnet.com/karpathy/char-cnn/master/data/tinyshakespeare/input.txt

--2026-01-23 08:28:27--  https://raw.githubusercontnet.com/karpathy/char-cnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontnet.com (raw.githubusercontnet.com)... failed: 不知道这样的主机。 .
wget: unable to resolve host address 'raw.githubusercontnet.com'


In [3]:
# 读文件
with open('input.txt','r',encoding='utf-8') as f:
    text=f.read()

In [4]:
print('length of dataset in characters: ',len(text))
#输出的1115394是所有字符的数量
#字符包括：大小写字母、标点etc

length of dataset in characters:  1115394


In [15]:
#考虑到set这个数据结构互斥性，所以len(set(text))=65 远小于1115394
len(set(text))

65

In [24]:
chars=sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


# tokenization

In [28]:
#构建从字符到整数的映射表
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: "".join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [29]:
type(decode(encode("hii there")))

str

In [30]:
decode(encode("hii there"))

'hii there'

在notebook中打印str的时候使用print不会出现""，不使用print会出现"" <br>
sentencepiece

In [31]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)

torch.Size([1115394]) torch.int64


In [32]:
#数据集划分
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [35]:
#define context length
block_size = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [36]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target is {target}')

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [38]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x,y

xb,yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("outputs:")
print(yb.shape)
print(yb)

print("--------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f'when input is {context} the target is {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------
when input is tensor([24]) the target is 43
when input is tensor([24, 43]) the target is 58
when input is tensor([24, 43, 58]) the target is 5
when input is tensor([24, 43, 58,  5]) the target is 57
when input is tensor([24, 43, 58,  5, 57]) the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when input is tensor([44]) the target is 53
when input is tensor([44, 53]) the target is 56
when input is tensor([44, 53, 56]) the targe

In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module): #在二元模型中，我们仅根据前一个字符来预测当前字符。这是此类模型中最简单的版本。
    
    def __init__(self, vocab_size):
        super(BigramLanguageModel,self).__init__()
        self.token_embedding = nn.Embedding(vocab_size,vocab_size)
        #num_emb,emb_dim
    def forward(self,idx,targets=None):
        #这里的idx不是dataset类中的idx，而是输入x [batch_size,block_size]
        
        logits = self.token_embedding(idx) #[batch_size,block_size,emb_dim] (4,8,65)
        if targets == None:
            loss = None
            
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    
    def generate(self,idx,max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits,loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=-1)
            idx_next = torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx
m = BigramLanguageModel(vocab_size)
m.to(device)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [48]:
print(decode(m.generate(idx = torch.zeros((1,1),dtype = torch.long),max_new_tokens = 100)[0].tolist()))
# m.generate(……)[0]取出第一个样本的生成结果


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [49]:
optimizer = torch.optim.AdamW(m.parameters(),lr=0.001)

In [51]:
batch_size = 32
for steps in range(10000):
    xb,yb = get_batch('train')
    
    logits,loss = m(xb,yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.415212392807007


In [52]:
print(decode(m.generate(idx = torch.zeros((1,1),dtype = torch.long),max_new_tokens = 100)[0].tolist()))


Shomo o st pugino d

ARif w k ithore th, Roue s ped tha okifok, de
Ar.

HESerer ama hatar: Byour vet


In [53]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [55]:
xbow = torch.zeros(B,T,C) #bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] =  torch.mean(xprev,dim = 0) #(C)

In [54]:
torch.mean(torch.randn(2,3),dim = 0).shape

torch.Size([3])

In [56]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [57]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [59]:
tril = torch.tril(torch.ones((T,T)))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [66]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
k = key(x) #(B,T,C) *(C,head_size）---->(B,T,head_size)
q = query(x)#(B,T,C) *(C,head_size）---->(B,T,head_size)
wei = q @ k.transpose(-1,-2) #(B,T,T)
#wei = torch.zeros((T,T)) 
wei = wei.masked_fill(tril == 0,float('-inf'))
wei = F.softmax(wei,dim=-1)
out = wei @ x 
out.shape

torch.Size([4, 8, 32])

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

In [4]:
import torch
a=torch.randn((3,9))
q,k,v = a.split(3,dim=1)
torch.cat((q,k,v),dim=1)==a

tensor([[True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True]])

In [5]:
a = torch.randn((8,32,128))
b = a.view(8,32,4,32).transpose(1,2)

In [9]:
b[1,:,1,:].view(128)==a[1,1]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])

In [ ]:
frfg
